In [1]:
!pip install pandas-datareader==0.10.0 pytrends basedosdados

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 14.7 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 108 kB 68.0 MB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
     |████████████████████████████████| 3.7 MB 69.4 MB/s 
     |████████████████████████████████| 133 kB 65.3 MB/s 
     |████████████████████████████████| 135 kB 72.2 MB/s 
     |████████████████████████████████| 25.5 MB 72.0 MB/s 
     |████████████████████████████████| 270 kB 62.4 MB/s 
     |████████████████████████████████| 58 kB 6.8 MB/s 
     |████████████████████████████████| 7.5 MB 66.4 MB/s 
     |████████████████████████████████| 70 kB 9.7 MB/s 
     |████████████████████████████████| 1.1 MB 62.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |██████████████████████

In [2]:
import matplotlib.dates as mdates      
import matplotlib.pyplot as plt    
from matplotlib import cm    
import seaborn as sns
import basedosdados as bd

import numpy as np
import pandas as pd
from pytrends.request import TrendReq                       
                    
import pandas_datareader.data as web                        
import datetime as dt                                      
from datetime import timedelta
%matplotlib inline

plt.style.use('ggplot')

FileNotFoundError: ignored

In [ ]:
# Para carregar o dado direto no pandas
pesquisa_eleitoral = bd.read_table(dataset_id='br_poder360_pesquisas',table_id='microdados',billing_project_id="eleicoes-361713")
pesquisa_eleitoral.head(3)

In [ ]:
pesquisa_eleitoral['data'] = pd.to_datetime(pesquisa_eleitoral['data'])

In [ ]:
cmap=cm.get_cmap('coolwarm')  

In [ ]:
#++++++++++++++++++++++ conexao com o servidor ++++++++++++++++++++++++++++  
pytrends=TrendReq(hl='en-US', tz=360)                                       
#++++++++++++++++++++++ lista com palavra-chave +++++++++++++++++++++++++++  
kw_list=['lula','bolsonaro','ciro','simone']            

In [ ]:
inicio=dt.datetime(2018,1,2)                                                 
fim=dt.datetime(2022,10,3)     

In [ ]:
## TREND SEARCH    
pytrends.build_payload(kw_list, cat=0,timeframe=f'{inicio.date()} {fim.date()}',geo='BR',gprop='')  
df=pytrends.interest_over_time() 
df = df[kw_list]
df.head(3)

In [ ]:
x_text = df.index[-1]+timedelta(days=2)

fig,_ = plt.subplots(figsize=(15,8))
plt.plot(df,linewidth=2.5)

plt.text(x=x_text,y=df['lula'].values[-1],s='Lula',fontsize=10,color='#E74C3C',weight='bold')  
plt.text(x=x_text,y=df['bolsonaro'].values[-1],s='Bolsonaro',fontsize=10,color='#2980B9',weight='bold')  

fig.tight_layout()

In [ ]:
pesquisa_eleitoral[pesquisa_eleitoral.nome_candidato.str.contains('José Serra')]

In [ ]:
# mask = (df.instituto=='Vox Populi')&(df.cargo == 'presidente')&(df.data.dt.month>=1)&(df.data.dt.month<=10)&(df.turno==2)
mask = (pesquisa_eleitoral.cargo == 'presidente')&(pesquisa_eleitoral.turno==1)&(pesquisa_eleitoral.nome_candidato.isin(['Bolsonaro','Lula','Marina','Dilma Rousseff','Aécio Neves']) )&(pesquisa_eleitoral.instituto.isin(['Ipec','Quaest','MDA','Ipespe','Datafolha','Ideia' 'Inteligência','FSB Pesquisa']))
eleicoes_presidente = pesquisa_eleitoral.loc[mask].groupby(['data','nome_candidato'])[['percentual']].agg(np.nanmean).reset_index()

In [ ]:
eleicoes_presidente.groupby([eleicoes_presidente.data.dt.year,'nome_candidato']).mean()

In [ ]:
pesquisa_eleitoral[pesquisa_eleitoral.instituto.str.contains('Atlas')]

In [ ]:
pesquisa_eleitoral[pesquisa_eleitoral.nome_candidato.str.contains('Soraya Thronicke')]

In [ ]:
pesquisa_eleitoral.loc[mask]

In [ ]:
mask = (pesquisa_eleitoral.nome_candidato.isin(['Bolsonaro','Lula']))&(pesquisa_eleitoral.data.dt.year==2022)&(pesquisa_eleitoral.instituto.isin(['AtlasIntel/Internet'])&(pesquisa_eleitoral.turno==1))
fig,_ = plt.subplots(figsize=(15,8))
sns.scatterplot(data=pesquisa_eleitoral.loc[mask], x="data", y="percentual", hue="nome_candidato")
sns.lineplot(data=pesquisa_eleitoral.loc[mask], x="data", y="percentual", hue="nome_candidato")
fig.tight_layout()